In [1]:
from keras.preprocessing import sequence
import keras
import tensorflow_datasets as tfds
import tensorflow as tf
import os
import numpy as np
import pandas as pd

In [2]:
textfile = ("C:/Users/wilde/Downloads/Python_Certification/house_script.txt")
data = open(textfile, 'rb').read().decode(encoding='utf-8')
print ('Length of text: {} characters'.format(len(textfile)))

Length of text: 62 characters


In [3]:
# Take a look at the first 250 characters in text
print(data[:250])

[Rebecca riding bus and then running into a school. Meets up with Melanie] Melanie: Why are you late? Rebecca: You’re not going to like the answer. Melanie: I already know the answer. Rebecca: I missed the bus. Melanie: I don’t doubt it, no bus stops


In [4]:
vocab = sorted(set(data))
# Creating a mapping from unique characters to indices
char2idx = {u:i for i, u in enumerate(vocab)}
idx2char = np.array(vocab)

def text_to_int(data):
  return np.array([char2idx[c] for c in data])

text_as_int = text_to_int(data)

In [5]:
# lets look at how part of our text is encoded
print("Text:", data[:15])
print("Encoded:", text_to_int(data[:15]))

Text: [Rebecca riding
Encoded: [47 40 53 50 53 51 51 49  0 66 57 52 57 62 55]


In [6]:
def int_to_text(ints):
  try:
    ints = ints.numpy()
  except:
    pass
  return ''.join(idx2char[ints])

print(int_to_text(text_as_int[:15]))

[Rebecca riding


In [7]:
seq_length = 100  # length of sequence for a training example
examples_per_epoch = len(data)//(seq_length+1)

# Create training examples / targets
char_dataset = tf.data.Dataset.from_tensor_slices(text_as_int)

In [8]:
sequences = char_dataset.batch(seq_length+1, drop_remainder=True)

In [9]:
def split_input_target(chunk):  # for the example: hello
    input_text = chunk[:-1]  # hell
    target_text = chunk[1:]  # ello
    return input_text, target_text  # hell, ello

dataset = sequences.map(split_input_target)  # we use map to apply the above function to every entry

In [10]:
for x, y in dataset.take(2):
  print("\n\nEXAMPLE\n")
  print("INPUT")
  print(int_to_text(x))
  print("\nOUTPUT")
  print(int_to_text(y))



EXAMPLE

INPUT
[Rebecca riding bus and then running into a school. Meets up with Melanie] Melanie: Why are you late

OUTPUT
Rebecca riding bus and then running into a school. Meets up with Melanie] Melanie: Why are you late?


EXAMPLE

INPUT
 Rebecca: You’re not going to like the answer. Melanie: I already know the answer. Rebecca: I missed

OUTPUT
Rebecca: You’re not going to like the answer. Melanie: I already know the answer. Rebecca: I missed 


In [11]:
BATCH_SIZE = 64
VOCAB_SIZE = len(vocab)  # vocab is number of unique characters
EMBEDDING_DIM = 256
RNN_UNITS = 1024

# Buffer size to shuffle the dataset
# (TF data is designed to work with possibly infinite sequences,
# so it doesn't attempt to shuffle the entire sequence in memory. Instead,
# it maintains a buffer in which it shuffles elements).
BUFFER_SIZE = 10000

data1 = dataset.shuffle(BUFFER_SIZE).batch(BATCH_SIZE, drop_remainder=True)

In [12]:
def build_model(vocab_size, embedding_dim, rnn_units, batch_size):
  model = tf.keras.Sequential([
    tf.keras.layers.Embedding(vocab_size, embedding_dim,
                              batch_input_shape=[batch_size, None]),
    tf.keras.layers.LSTM(rnn_units,
                        return_sequences=True,
                        stateful=True,
                        recurrent_initializer='glorot_uniform'),
    tf.keras.layers.Dense(vocab_size)
  ])
  return model

model = build_model(VOCAB_SIZE,EMBEDDING_DIM, RNN_UNITS, BATCH_SIZE)
model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding (Embedding)        (64, None, 256)           20736     
_________________________________________________________________
lstm (LSTM)                  (64, None, 1024)          5246976   
_________________________________________________________________
dense (Dense)                (64, None, 81)            83025     
Total params: 5,350,737
Trainable params: 5,350,737
Non-trainable params: 0
_________________________________________________________________


In [13]:
for input_example_batch, target_example_batch in data1.take(1):
  example_batch_predictions = model(input_example_batch)  # ask our model for a prediction on our first batch of training data (64 entries)
  print(example_batch_predictions.shape, "# (batch_size, sequence_length, vocab_size)")  # print out the output shape

(64, 100, 81) # (batch_size, sequence_length, vocab_size)


In [14]:
# we can see that the predicition is an array of 64 arrays, one for each entry in the batch
print(len(example_batch_predictions))
print(example_batch_predictions)

64
tf.Tensor(
[[[ 7.95843545e-04 -3.62348044e-03 -2.10069958e-03 ...  1.14218204e-03
    3.13831260e-05  1.44786853e-03]
  [ 2.99980864e-03 -4.65280795e-03 -5.43536525e-03 ...  4.02377266e-03
   -7.22963596e-05 -3.20299226e-03]
  [-3.47720133e-03 -2.42220471e-03 -3.89663130e-03 ...  7.25093298e-04
    7.99313560e-03 -6.91624964e-03]
  ...
  [-3.92452069e-03  3.18783708e-03  7.39302114e-03 ...  2.04230286e-03
   -3.64290783e-04 -4.62318677e-03]
  [ 2.35126214e-03 -3.02760862e-04  7.53474073e-04 ...  2.35056155e-03
   -4.64338902e-03 -8.32307246e-03]
  [ 3.06645408e-03  2.19218596e-03  6.70239097e-03 ...  4.80801146e-03
   -6.46725995e-03 -6.21082634e-03]]

 [[ 1.84395595e-03  1.35494908e-03  6.16411585e-03 ...  2.76800175e-03
   -2.45373906e-03  8.95111531e-04]
  [-4.60220873e-03  3.13576311e-03  4.66461666e-03 ...  9.40705766e-04
    1.78144418e-03  6.42099534e-04]
  [-4.93489392e-03  3.83698614e-03 -1.85017334e-03 ... -2.41430919e-03
    3.94137483e-03  7.12798268e-04]
  ...
  [ 4.069

In [15]:
# lets examine one prediction
pred = example_batch_predictions[0]
print(len(pred))
print(pred)
# notice this is a 2d array of length 100, where each interior array is the prediction for the next character at each time step

100
tf.Tensor(
[[ 7.9584355e-04 -3.6234804e-03 -2.1006996e-03 ...  1.1421820e-03
   3.1383126e-05  1.4478685e-03]
 [ 2.9998086e-03 -4.6528080e-03 -5.4353653e-03 ...  4.0237727e-03
  -7.2296360e-05 -3.2029923e-03]
 [-3.4772013e-03 -2.4222047e-03 -3.8966313e-03 ...  7.2509330e-04
   7.9931356e-03 -6.9162496e-03]
 ...
 [-3.9245207e-03  3.1878371e-03  7.3930211e-03 ...  2.0423029e-03
  -3.6429078e-04 -4.6231868e-03]
 [ 2.3512621e-03 -3.0276086e-04  7.5347407e-04 ...  2.3505616e-03
  -4.6433890e-03 -8.3230725e-03]
 [ 3.0664541e-03  2.1921860e-03  6.7023910e-03 ...  4.8080115e-03
  -6.4672600e-03 -6.2108263e-03]], shape=(100, 81), dtype=float32)


In [16]:
# and finally we'll look at a prediction at the first timestep
time_pred = pred[0]
print(len(time_pred))
print(time_pred)
# and of course its 65 values representing the probabillity of each character occuring next

81
tf.Tensor(
[ 7.9584355e-04 -3.6234804e-03 -2.1006996e-03  1.1938311e-03
  2.4589216e-03 -4.2584562e-04  1.4638313e-03  5.2954094e-04
 -1.3516761e-03 -5.3014033e-03  7.4480437e-03  2.4958833e-03
  1.9846698e-03 -1.1027359e-03 -5.1027369e-03  2.4702069e-03
  4.9059279e-03 -4.1747792e-03 -4.0667178e-03  4.8114164e-03
 -3.0620345e-03 -7.5250177e-04  4.7471281e-04  2.2397088e-03
  4.0121530e-03  6.0209753e-03  1.7075054e-03 -2.7557034e-03
 -3.9565428e-03  8.6266664e-04 -2.0906148e-03  1.7474452e-04
  1.7439579e-03 -1.2666560e-04  2.8046418e-03 -1.0083038e-03
  2.8730023e-03 -2.8789132e-03  3.8023510e-03 -2.6491205e-03
 -9.1494238e-03  2.4093529e-03  1.0459394e-03 -4.6120677e-03
  1.6326011e-03 -2.9001390e-03  3.5816079e-04 -2.8459455e-03
  5.2289572e-03  2.1124189e-03 -6.0603663e-04 -2.2824600e-03
 -4.1601271e-04  4.1880063e-03 -5.5222790e-04  9.7019860e-04
  6.3097542e-03 -3.7836288e-03  2.0662448e-03 -4.1568666e-03
 -3.4592720e-03 -2.3819541e-03  2.3615044e-04 -5.0616213e-03
 -4.623108

In [17]:
# If we want to determine the predicted character we need to sample the output distribution (pick a value based on probabillity)
sampled_indices = tf.random.categorical(pred, num_samples=1)

# now we can reshape that array and convert all the integers to numbers to see the actual characters
sampled_indices = np.reshape(sampled_indices, (1, -1))[0]
predicted_chars = int_to_text(sampled_indices)

predicted_chars  # and this is what the model predicted for training sequence 1

"}gtBJVRPS.--9(3ryyWA$U1,R'jyOuASMfEJyfLtJDJEA(NyG*A6iWa?URPNWR;,Y}a-acg]!“ITHCzjY1HbGl'RJbxEzb(kM:Eo"

In [18]:
def loss(labels, logits):
  return tf.keras.losses.sparse_categorical_crossentropy(labels, logits, from_logits=True)

In [19]:
model.compile(optimizer='adam', loss=loss)

In [20]:
# Directory where the checkpoints will be saved
checkpoint_dir = './training_checkpoints'
# Name of the checkpoint files
checkpoint_prefix = os.path.join(checkpoint_dir, "ckpt_{epoch}")

checkpoint_callback=tf.keras.callbacks.ModelCheckpoint(
    filepath=checkpoint_prefix,
    save_weights_only=True)

In [30]:
history = model.fit(data1, epochs=50, callbacks=[checkpoint_callback])

Epoch 1/50
1/6 [====>.........................] - ETA: 45s - loss: 4.3940

KeyboardInterrupt: 

In [41]:
model = build_model(VOCAB_SIZE, EMBEDDING_DIM, RNN_UNITS, batch_size=1)

In [42]:
model.load_weights(tf.train.latest_checkpoint(checkpoint_dir))
model.build(tf.TensorShape([1, None]))

In [44]:
def generate_text(model, start_string):
  # Evaluation step (generating text using the learned model)

  # Number of characters to generate
  num_generate = 800

  # Converting our start string to numbers (vectorizing)
  input_eval = [char2idx[s] for s in start_string]
  input_eval = tf.expand_dims(input_eval, 0)

  # Empty string to store our results
  text_generated = []

  # Low temperatures results in more predictable text.
  # Higher temperatures results in more surprising text.
  # Experiment to find the best setting.
  temperature = 1.0

  # Here batch size == 1
  model.reset_states()
  for i in range(num_generate):
      predictions = model(input_eval)
      # remove the batch dimension
    
      predictions = tf.squeeze(predictions, 0)

      # using a categorical distribution to predict the character returned by the model
      predictions = predictions / temperature
      predicted_id = tf.random.categorical(predictions, num_samples=1)[-1,0].numpy()

      # We pass the predicted character as the next input to the model
      # along with the previous hidden state
      input_eval = tf.expand_dims([predicted_id], 0)

      text_generated.append(idx2char[predicted_id])

  return (start_string + ''.join(text_generated))

In [45]:
inp = input("Type a starting string: ")
print(generate_text(model, inp))

Type a starting string:  House:


House: Thes the fakin? Wilson: Whoud toproum? Loo moin conied sasta days. Oh, wich dnytchun. Cameron: House: Fore andy a gont geys! Ay then a dovy a fitht a dottere. It’s mannitspitaimactert. Cardnay: [guy ceo andy Rebecca rom. House: Will meor wilks voonk fitha jay wains foazont. Herewan: No jakey! Famenong you camentism but innoum the kilsow diy gother masting] [lduy. Wils ifing frithan a dizuct? Coferon: Pat the pli-trengey joom she foupes. Comyoul-an’t (twe lay Jadicue warks. Se mickene. Rabecco: [Gay bruttors heve? I’s battling. Camy: Eximedon it you bucteriny Fordice hareset. House: You ord, andictlicishat to ghe lig tod juin… whils Sicto. ,owh on Miaver wolds. Cmase: Ye, tomeroute butt cawh thays fore te furst onfavimod in a it’s lime thellioge stasin. [SV he’r for. Ctheme near his thiw r
